more interesting example. it passes the whole paper and context to optimizer

In [ ]:
import sys

sys.path.append("C:/Users/vansari/Documents/PhotonicAI")
import pandas as pd
import textgrad as tg
from dotenv import load_dotenv

load_dotenv(dotenv_path="../../.env")

# tg.set_backward_engine("gpt-4o", override=True)
llm_engine = tg.get_engine("gpt-4o")
tg.set_backward_engine(llm_engine, override=True)


# Step 1: Get an initial response from an LLM.
df = pd.read_parquet("../papers/db/AMF_papers.parquet")
idx = 223
article = df.loc[idx]["text_full"]
print("--> filename: ", df.loc[idx]["filename"])
# 244, 223,

problem_text = f"""Is this a single academic article, and not a dissertation or collection of papers (single_article)?
Is the main topic of this article about integrated photonic circuits (topic_photonic)?
If yes, find the photonic components that are used on the chip.
Return a concise list of these photonic components, if any (components_list).
For each component, try to extract: brief spec,
and the number of optical input (N) and output (M) ports denoted by NxM, e.g. 1x2.
Do not parse specifications and descriptive modifiers of a component as separate components.
Finally, is there an enough information to understand and desrcibe how the on-chip components
are interconnected to form the photonic circuit (circuit_complete)?
Answer in YAML following the template:
single_article: True/False
topic_photonic: True/False
components_list:
  - a 1x1 modulator with MHz speed
  - a 1x2 component ...
  ...
circuit_complete: True/False


INPUT ARTICLE:
{article}
"""

problem = tg.Variable(
    problem_text, role_description="the parsing problem", requires_grad=False
)

model = tg.BlackboxLLM("gpt-4o")
code = model(problem)

print(30 * "=" + "INITIAL ANSWER")
print(code)
print(40 * "=")
print(40 * "=")

code.set_role_description("The yaml code to optimize")
code.requires_grad = True

photonic_critic_prompt = """You are a smart language model expert in photonic integrated circuits.
This YAML file should be an accurate summary of the photonic components presented in the input article.
Evaluate components_list in YAML based on:
- this should be only a list of photonic components.
- it should only represent photonic components on the chip, and not off the chip.
- all detail specifications and description of a component should be included for each item.
- detail specifications and descriptive modifiers of a component should not be parsed as separate components.
- are any photonic components missing?
- does YAML follow the provided template?
Also evaluate circuit_complete boolean:
- is it correct? can you understand and describe the connection between items in components_list?
You do not propose a new YAML file, only evaluate the existing YAML file critically and give very concise feedback."""
loss_system_prompt = tg.Variable(
    photonic_critic_prompt,
    requires_grad=False,
    role_description="system prompt to the loss function",
)


format_string = """Problem: {problem}\nCurrent YAML code: {code}"""

fields = {"problem": None, "code": None}
formatted_llm_call = tg.autograd.FormattedLLMCall(
    engine=llm_engine,
    format_string=format_string,
    fields=fields,
    system_prompt=loss_system_prompt,
)


# Finally, the loss function
def loss_fn(problem: tg.Variable, code: tg.Variable) -> tg.Variable:
    inputs = {"problem": problem, "code": code}

    return formatted_llm_call(
        inputs=inputs,
        response_role_description=f"evaluation of the {code.get_role_description()}",
    )


loss = loss_fn(problem, code)
print(30 * "=" + "LOSS")
print(loss.value)
loss.backward()

print(30 * "=" + "CODE GRAD")
print(code.gradients)

optimizer = tg.TGD(parameters=[code])
optimizer.step()
print(30 * "=" + "UPDATED CODE")
print(code.value)
print(30 * "=")
code

In [ ]:
loss.generate_graph()

In [ ]:
optimizer.zero_grad()
loss = loss_fn(problem, code)
loss.backward()
optimizer.step()
print(code.value)

In [ ]:
optimizer.zero_grad()
loss = loss_fn(problem, code)
loss.backward()
optimizer.step()
print(code.value)

In [ ]:
optimizer.zero_grad()
loss = loss_fn(problem, code)
loss.backward()
optimizer.step()
print(code.value)

In [ ]:
optimizer.zero_grad()
loss = loss_fn(problem, code)
loss.backward()
optimizer.step()
print(code.value)